# Fractal exploration with Plotly

In [1]:
%%capture
from tqdm import notebook as tqdm
tqdm.tqdm()

import time
import datetime

import ipywidgets as widgets
from ipywidgets import interact, interactive, fixed, interact_manual

import plotly.express as px
import plotly.graph_objects as go

import numpy
import pandas as pd
import math

import PIL.Image
import imageio

# Burning Ship and  Mandelbrot Skews

In [2]:
def BuringShip(x,y,power,shift, Yrange,Xrange): 
    zy = y * (Yrange[1] - Yrange[0]) / (imagesize - 1)  + Yrange[0]
    zx = x * (Xrange[1] - Xrange[0]) / (imagesize - 1)  + Xrange[0] 
    z = zx + zy * 1j
    c = z 
    for i in range(max_it):
        if abs(z) > 2.0:
            return exterior(z,i)

        z = (abs(z.real) + abs(z.imag)* 1j) ** power + c +shift
    return interior(z,i)


def MandelbrotSkew(px,py,power,shift, Yrange,Xrange): 
    zy = (py * (Yrange[1] - Yrange[0]) / (imagesize))  + Yrange[0]
    zx = (px * (Xrange[1] - Xrange[0]) / (imagesize))  + Xrange[0]
    z = zx + zy * 1j
    c = z
    for i in range(max_it):
        if abs(z) > 2.0:
            return exterior(z,i)

        z = z ** power + c +shift

    return interior(z,i)

def Newton(px,py,power,shift, Yrange,Xrange): 
    zy = (py * (Yrange[1] - Yrange[0]) / (imagesize))  + Yrange[0]
    zx = (px * (Xrange[1] - Xrange[0]) / (imagesize))  + Xrange[0]
    z = zx + zy * 1j
    c = z
    for i in range(max_it):
        if abs(z) > 2.0:
            return exterior(z,i)
        try:
            z = z - (z*z*z-c)/(3*z*z)
        except:
            z=z
    return interior(z,i)


def Nova(px,py,power,shift, Yrange,Xrange): 
    zy = (py * (Yrange[1] - Yrange[0]) / (imagesize))  + Yrange[0]
    zx = (px * (Xrange[1] - Xrange[0]) / (imagesize))  + Xrange[0]
    z = zx + zy * 1j
    c = z
    for i in range(max_it):
        if abs(z) > 2.0:
            return exterior(z,i)
        try:
            z = z - (z-1)**3 / (3*z*z) +c
        except:
            z=z
    return interior(z,i)

def Magnet(px,py,power,shift, Yrange,Xrange): 
    zy = (py * (Yrange[1] - Yrange[0]) / (imagesize))  + Yrange[0]
    zx = (px * (Xrange[1] - Xrange[0]) / (imagesize))  + Xrange[0]
    z = zx + zy * 1j
    c = z
    for i in range(max_it):
        if abs(z) > 2.0:
            return exterior(z,i)
        try:
            z =  ((z*z +c -1) / (2*z + c - 2))**(0.5)
        except:
            z=z
    return interior(z,i)


fractals = [BuringShip, MandelbrotSkew, Newton, Nova, Magnet]


def interior(z,i): 
    return i
    
def exterior(z,i):
    return i

def iteration_count(z,i):
    return i

def orbit_distance(z,i):
    return abs(z)

def orbit_distance100x(z,i):
    return abs(z)*100

def orbit_distance_log(z,i):
    try:
        return math.log(abs(z))
    except:
         return math.log(abs(z)+1)

def orbit_distance_sinh(z,i):
    try:
        return math.sinh(abs(z))
    except:
        return math.sinh(abs(z)+1)
        
maps = [iteration_count, orbit_distance, orbit_distance100x, orbit_distance_log, orbit_distance_sinh ]

In [3]:
def browse_fractals(fractals):
    def fractalpick(fractal):

        global Frac
        Frac = fractal
        return print(fractal.__name__)
    fractal =  interact(fractalpick, fractal=fractals)

# pick fractal    
browse_fractals(fractals)


def powerset(Power):
    global power
    power=Power
    return
    
def shiftset(Shift):
    global shift
    shift=Shift
    return

def exteriorset(Exterior):
    global exterior
    exterior=Exterior

def interiorset(Interior):
    global interior
    interior=Interior
    
    
# pick fractal parameters 
interact(powerset, Power=2.0, description="Power");
interact(shiftset, Shift=0.0, description="shift");

interact(interiorset, Interior=maps);
interact(exteriorset, Exterior=maps);

interactive(children=(Dropdown(description='fractal', options=(<function BuringShip at 0x0000019D8B4C76A8>, <f…

interactive(children=(FloatSlider(value=2.0, description='Power', max=6.0, min=-2.0), Output()), _dom_classes=…

interactive(children=(FloatSlider(value=0.0, description='Shift', max=1.0), Output()), _dom_classes=('widget-i…

interactive(children=(Dropdown(description='Interior', options=(<function iteration_count at 0x0000019D8B4CF51…

interactive(children=(Dropdown(description='Exterior', options=(<function iteration_count at 0x0000019D8B4CF51…

# Display

In [4]:
# get zoom from last plot or start at ((-1,-2), (1,2)) if figure doesn't exist
if "figzoom" in globals():
    Xrange=figzoom.layout.xaxis.range 
    Yrange=figzoom.layout.yaxis.range 
    
else:
    Xrange = (-1.0, 1.0)
    Yrange = (-2, 2.0)
    
# set the size of the array/image and interation depth
imagesize = 1080
max_it =256
array = numpy.zeros(shape=(imagesize,imagesize)) 


for i in tqdm.tqdm(range(imagesize)):
    for j in range(imagesize):
        array[i,j] = Frac(i, j, power, shift ,Xrange ,Yrange) 

figzoom = go.FigureWidget(data = 
                go.Heatmap(
                    z=array, 
                    zsmooth='best',
                    showscale=False,
                    colorscale='icefire',
                    x0=Xrange[0], dx=(Xrange[1] - Xrange[0]) / (imagesize),
                    y0=Yrange[0], dy=(Yrange[1] - Yrange[0]) / (imagesize),
                ),
                      layout =
                go.Layout( autosize=False, width=1080, height=1080, 
                    yaxis={'visible': False, 'showticklabels': False},
                    xaxis={'visible': False, 'showticklabels': False},
                    margin=dict(l=0, r=0, b=0,t=0,pad=1))
               )
figzoom


FigureWidget({
    'data': [{'colorscale': [[0.0, '#000000'], [0.0625, '#001f4d'], [0.125,
                   …

In [ ]:
1000/1000 [00:09<00:00, 104.48it/s]
1000/1000 [00:08<00:00, 115.97it/s]
1080/1080 [00:20<00:00, 52.03it/s]
2000/2000 [00:35<00:00, 56.73it/s]
4000/4000 [02:20<00:00, 28.42it/s]

In [33]:
#reset zoom
del figzoom

# Histogram of pixel values

In [28]:
def pixelsample(numpyarray):
    flatarray = numpyarray.flatten()
    sample = numpy.random.choice(flatarray, size=1000)
    return sample

@interact
def transform(lower=(0,255,1),
              lower_shift=0,
              lower_scale=1,
              lower_power=1,
              higher=255,
              higher_shift=0,
              higher_scale=(-1,1,0.01),
              higher_power=1):
    global array2
    array2=array.copy()
    
    array2[array2 < lower] = (array2[array2 < lower]+ lower_shift )*lower_scale**lower_power

    array2[array2 > higher] = (array2[array2 > higher]+ higher_shift )*higher_scale**higher_power
    
    fig = go.Figure(data=go.Violin(x=pixelsample(array), name="raw",side='positive',orientation='h'))
    fig.add_trace(go.Violin(x=pixelsample(array2), name="transformed",side='positive',orientation='h'))
    
    
    return fig.show()

interactive(children=(IntSlider(value=127, description='lower', max=255), IntSlider(value=0, description='lowe…

In [49]:
figshift = go.FigureWidget(data = 
                go.Heatmap(
                    z=array2, 
                    zsmooth='best',
                    showscale=False,
                    colorscale='icefire',
                    x0=Xrange[0], dx=(Xrange[1] - Xrange[0]) / (imagesize),
                    y0=Yrange[0], dy=(Yrange[1] - Yrange[0]) / (imagesize),
                ),
                      layout =
                go.Layout( autosize=False, width=1080, height=1080, 
                    yaxis={'visible': False, 'showticklabels': False},
                    xaxis={'visible': False, 'showticklabels': False},
                    margin=dict(l=0, r=0, b=0,t=0,pad=1))
               )
figshift




FigureWidget({
    'data': [{'colorscale': [[0.0, '#000000'], [0.0625, '#001f4d'], [0.125,
                   …

In [31]:
now = str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
figshift.write_image(name_image(figzoom)+ now +".jpeg")

## Export as html

In [7]:
figzoom.write_html("burningship.html")

## Save as image

In [14]:
# image naming function
def name_image(fig):

    namestring = Frac.__name__
    namestring += "^" +str(power)
    namestring += "+" +str(shift)

    xmin,xmax =figzoom.layout.xaxis.range
    ymin,ymax =figzoom.layout.yaxis.range
    namestring +="({:.4f},{:.4f}),({:.4f},{:.4f})".format(xmin,xmax,ymin,ymax)
    namestring += exterior.__name__
    namestring += interior.__name__
    
    return namestring

In [10]:
now = str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
figzoom.write_image(name_image(figzoom)+ now +".jpeg")

## Save as high res image

In [21]:
#Hi rez image
Xrange=figzoom.layout.xaxis.range 
Yrange=figzoom.layout.yaxis.range
# set the size of the array/image and interation depth
imagesize = 2160
max_it =256
array = numpy.zeros(shape=(imagesize,imagesize)) 

for i in tqdm.tqdm(range(imagesize)):
    for j in range(imagesize):
        array[i,j] = Frac(i, j, power, shift, Xrange, Yrange)

Hires = go.FigureWidget(data = 
                go.Heatmap(
                    z=array, 
                    zsmooth='best',
                    showscale=False,
                    colorscale='icefire',
                    x0=Xrange[0], dx=(Xrange[1] - Xrange[0]) / (imagesize),
                    y0=Yrange[0], dy=(Yrange[1] - Yrange[0]) / (imagesize),
                ),
                      layout =
                go.Layout( autosize=False, width= 2160, height= 2160, 
                    yaxis={'visible': False, 'showticklabels': False},
                    xaxis={'visible': False, 'showticklabels': False},
                    margin=dict(l=0, r=0, b=0,t=0,pad=1))
               )


now = str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
Hires.write_image(name_image(Hires) + now + ".jpeg")

del Hires

# Zoom in 

In [7]:
def fractal(imagesize,xrange, yrange):
    max_it =256
    array = numpy.zeros(shape=(imagesize,imagesize)) 


    for i in tqdm.tqdm(range(imagesize)):
        for j in range(imagesize):
            array[i,j] = Frac(i, j, power, shift ,xrange ,yrange) 

    fractalfig = go.FigureWidget(data = 
                go.Heatmap(
                    z=array, 
                    zsmooth='best',
                    showscale=False,
                    colorscale='icefire',
                    x0=xrange[0], dx=(xrange[1] - xrange[0]) / (imagesize),
                    y0=yrange[0], dy=(yrange[1] - yrange[0]) / (imagesize),
                ),
                      layout =
                go.Layout( autosize=False, width=imagesize, height=imagesize, 
                    yaxis={'visible': False, 'showticklabels': False},
                    xaxis={'visible': False, 'showticklabels': False},
                    margin=dict(l=0, r=0, b=0,t=0,pad=1))
               )
    
    
    
    return fractalfig.to_image("png")

render_path = "D:/Users/ben/Dropbox/aiart/vidout/"


def zoom(imagesize, startx, starty, endx, endy, steps):
    now = str(datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    writer = imageio.get_writer(render_path+'vidout' + now + '.mp4', fps=10)
    ## interpolation
    
    xranges = numpy.linspace(startx[0], endx[0], steps),  numpy.geomspace(startx[1], endx[1], steps)
    yranges = numpy.linspace(starty[0], endy[0], steps),  numpy.geomspace(starty[1], endy[1], steps)
    
    for step in tqdm.tqdm(range(steps)):
        
        image = fractal(imagesize,(xranges[0][step],xranges[1][step]),(yranges[0][step],yranges[1][step]))
        writer.append_data(imageio.imread(image))

    writer.close()

    print(render_path+'vidout' + now + '.mp4')

In [8]:
imagesize = 512
zoom(imagesize,(-1.0, 1.0),(-2.0, 2.0),figzoom.layout.xaxis.range, figzoom.layout.yaxis.range, 30)

D:\Users\ben__\Anaconda3\envs\tensorflow-gpu-2.0\lib\site-packages\numpy\core\function_base.py:410: RuntimeWarning:

invalid value encountered in log10





D:/Users/ben/Dropbox/aiart/vidout/vidout20200916-164703.mp4


# Multiprocessing 

In [5]:
import multiprocess
import numpy as np
from multiprocess import Array, Pool
from multiprocess import sharedctypes
import itertools


def MandelbrotMP(arg):
    px,py,power,shift, Yrange,Xrange,imagesize,max_it = arg
    zy = (py * (Yrange[1] - Yrange[0]) / (imagesize))  + Yrange[0]
    zx = (px * (Xrange[1] - Xrange[0]) / (imagesize))  + Xrange[0]
    z = zx + zy * 1j
    c = z
    for i in range(max_it):
        if abs(z) > 2.0:
            return i

        z = z ** power + c +shift
    return i

In [12]:
# get zoom from last plot or start at ((-1,-2), (1,2)) if figure doesn't exist
if "figzoomMP" in globals():
    Xrange=figzoomMP.layout.xaxis.range 
    Yrange=figzoomMP.layout.yaxis.range 
    
else:
    Xrange = (-1.0, 1.0)
    Yrange = (-2, 2.0)
    
# set the size of the array/image and interation depth
imagesize = 2000
max_it =256

array = numpy.zeros(shape=(imagesize,imagesize))
p = Pool(5)
for j in tqdm.tqdm(range(imagesize)):
        arguments = zip(list(range(imagesize)), 
                itertools.repeat(j), 
                itertools.repeat(power),
                itertools.repeat(shift) , 
                itertools.repeat(Xrange) ,
                 itertools.repeat(Yrange),
                 itertools.repeat(imagesize),
                 itertools.repeat(256))
        
        res = p.map(MandelbrotMP, arguments)
        array[:,j] = res
        
figzoomMP = go.FigureWidget(data = 
                go.Heatmap(
                    z=array, 
                    zsmooth='best',
                    showscale=False,
                    colorscale='icefire',
                    x0=Xrange[0], dx=(Xrange[1] - Xrange[0]) / (imagesize),
                    y0=Yrange[0], dy=(Yrange[1] - Yrange[0]) / (imagesize),
                ),
                      layout =
                go.Layout( autosize=False, width=1080, height=1080, 
                    yaxis={'visible': False, 'showticklabels': False},
                    xaxis={'visible': False, 'showticklabels': False},
                    margin=dict(l=0, r=0, b=0,t=0,pad=1))
               )
figzoomMP

FigureWidget({
    'data': [{'colorscale': [[0.0, '#000000'], [0.0625, '#001f4d'], [0.125,
                   …

In [ ]:
MP:
4000/4000 [04:00<00:00, 16.64it/s]
2000/2000 [01:08<00:00, 29.30it/s]
2000/2000 [01:10<00:00, 28.28it/s]
2000/2000 [01:27<00:00, 22.74it/s]
2000/2000 [01:12<00:00, 27.65it/s]
1080/1080 [00:27<00:00, 38.90it/s]
1000/1000 [00:21<00:00, 45.72it/s]
1000/1000 [00:22<00:00, 44.94it/s]

MP pool(1):
2000/2000 [01:27<00:00, 22.94it/s]

MP pool(4):
2000/2000 [01:23<00:00, 24.06it/s]

MP pool(5):
2000/2000 [01:10<00:00, 28.32it/s]